In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import read, write
from scipy import signal
from sklearn.decomposition import NMF
from sklearn.preprocessing import MinMaxScaler
from numpy.linalg import inv
import torch
import torch.optim as optim
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as I
from torch.nn import Sigmoid


from torch import transpose
from sklearn.neural_network import BernoulliRBM
import seaborn as sns
import warnings
warnings.simplefilter('ignore')
import numpy as np
from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt

In [2]:
from helpers import *
#from nmf_utils import *
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ffmpeg -i  "/content/drive/MyDrive/vocal_10.mp3" "vocal.wav";
!ffmpeg -i  "/content/drive/MyDrive/piano_10.mp3" "music.wav";

In [4]:
path_music  = "music.wav"
path_speech = "vocal.wav"
sec = 1088
music,speech_t,samplerate = load_data(path_music,path_speech,sec)

Shape of the music 47980800
Length : 1088.00s
Sample rate : 44100
Shape of the speech 47980800
Length : 1088.00s
Sample rate : 44100


In [5]:
WINDOW = 'hamming'
WINDOW_SIZE=480
OVERLAP = 0.6 * WINDOW_SIZE
NFFT=512

In [6]:
from scipy import signal

In [7]:
rate = 44100/16000
speech_t = signal.resample(speech_t,int(speech_t.shape[0]/rate))
music=signal.resample(music,int(music.shape[0]/rate))
samplerate_m=int(samplerate/rate)
samplerate_s=samplerate_m
samplerate_t=samplerate_m

In [8]:
samplerate_t

16000

# NMF

# Train / Validation

In [9]:
speech = butter_lowpass_filter(speech_t,4000,16000)

In [10]:
train_speech   = np.hstack((speech[samplerate_t*60:samplerate_t*440] , speech[samplerate_t*723:samplerate_t*930]))
train_music    = np.hstack((music[samplerate_t*60:samplerate_t*440] ,  music[samplerate_t*723:samplerate_t*930]))

test_speech ,test_music =  speech[samplerate_t*580:samplerate_t*700]  , music[samplerate_t*580:samplerate_t*700]

In [11]:
test_speech.shape, test_music.shape

((1920000,), (1920000,))

## Test different configurations 5,8,10,32 components

In [ ]:
B_all_list= []
for c in [10]:
  B_mixed = training(train_music,
               train_speech,
               s_component = c,
               m_component = c,
               iterations = 100,
               s_alpha = 0,
               m_alpha = 0,
               p = 2,
               init="random")
  B_mixed_new = MinMaxScaler().fit_transform(B_mixed)
  B_all_list.append(B_mixed_new)

# Validation

In [12]:
B_mixed = np.load("b_train.npy")
my_dict_2= {}
info_dict_2 = {} 
for s in [-5]:
  smr,sdr_speech,sdr_music,spectrum_s1,spectrum_s2,Y_mag = validation(B_mixed,test_speech, ## n component = 8 +8
                  test_music,
                  smr=s,
                  iterations=100,
                  m_component=8,
                  s_component=8,
                  alpha=0,
                  init="random"
)
  
  my_dict_2["SMR = {}".format(smr)] = info_dict_2 = ["SDR Speech = {:.2f} SDR Music = {:.2f}".format(sdr_speech,sdr_music)]


SMR = -5.00


KeyboardInterrupt: ignored

# RBM Training

In [20]:
from helpers_rbm import *
def train_layer( visible_size,hidden_size,      # this function allows us to train every layer separately, considering new dataloader that will feed into the new visible layer
                 s1,s2,
                 learning_rate,
                 epochs=2,
                 batch_size=256
                ):
  model = RBM(visible_size, hidden_size)        # Declare the model
  dataset=AudioDataset(Y_mag_scaled, s1, s2)   # Dataloader
  train_loader = DataLoader(dataset, 
                            batch_size=batch_size,
                            shuffle=False, 
                            num_workers=0)
  training_rbm(model, train_loader,learning_rate,n_epochs=epochs)      ## Train the layer according to its inputs ( see argument of the function "s1,s2")

  h_1,_,_ = model(torch.from_numpy(s1).float())     ## Inference of spectra magnitude of source 1
  h_2,_,_ = model(torch.from_numpy(s2).float())     ## Inference of spectra magnitude of source 1

  return model,h_1.cpu().detach().numpy(),h_2.cpu().detach().numpy()



In [21]:
# load training data in batches
from sklearn.preprocessing import MinMaxScaler 

s1_mag = np.abs(spectrum_s1).astype("float32")
s2_mag = np.abs(spectrum_s2).astype("float32")

s1_mag_scaled = MinMaxScaler().fit_transform(s1_mag).reshape(-1,257)
s2_mag_scaled = MinMaxScaler().fit_transform(s2_mag).reshape(-1,257)
Y_mag_scaled = MinMaxScaler().fit_transform(Y_mag).reshape(-1,257)


dataset=AudioDataset(Y_mag_scaled, s1_mag_scaled, s2_mag_scaled)

print('Number of samples in the audio is {}'.format(len(dataset)))

### First dataloader
batch_size = 256
train_loader = DataLoader(dataset, 
                          batch_size=batch_size,
                          shuffle=False, 
                          num_workers=0)

Number of samples in the audio is 10001


In [25]:
n_epochs = 150
bs = 128
lr = 0.001

## Training

In [26]:
model_1, h1_1, h_2_1 = train_layer(257,
                                   100,
                                   s1_mag_scaled,
                                   s2_mag_scaled,
                                   learning_rate = lr,
                                   epochs=n_epochs,
                                   batch_size=bs)

h1,x_reconstructed_1,h_reconstructed_1 = model_1(torch.from_numpy(s1_mag_scaled).float())
torch.save(model_1.state_dict(), "w_layer_1")
del model_1


Begin Training.....
epochs 0 MSE Loss 0.24621683359146118
epochs 10 MSE Loss 0.09550449252128601
Epoch    13: reducing learning rate of group 0 to 9.9990e-04.
epochs 20 MSE Loss 0.042575813829898834
Epoch    25: reducing learning rate of group 0 to 9.9980e-04.
epochs 30 MSE Loss 0.020238444209098816
Epoch    37: reducing learning rate of group 0 to 9.9970e-04.
epochs 40 MSE Loss 0.009981383569538593
Epoch    49: reducing learning rate of group 0 to 9.9960e-04.
epochs 50 MSE Loss 0.005052944645285606
Epoch    61: reducing learning rate of group 0 to 9.9950e-04.
epochs 60 MSE Loss 0.002662060782313347
epochs 70 MSE Loss 0.001482380204834044
Epoch    73: reducing learning rate of group 0 to 9.9940e-04.
epochs 80 MSE Loss 0.0009057027054950595
Epoch    85: reducing learning rate of group 0 to 9.9930e-04.
epochs 90 MSE Loss 0.0006265032570809126
Epoch    97: reducing learning rate of group 0 to 9.9920e-04.
epochs 100 MSE Loss 0.0004963790997862816
Epoch   109: reducing learning rate of grou

In [28]:
model_2, h1_2,h_2_2 = train_layer(100,
                                  50,
                                  h1_1,
                                  h_2_1,
                                  epochs=n_epochs,
                                  learning_rate = 0.001,
                                  batch_size=bs)
torch.save(model_2.state_dict(), "w_layer_2")
del model_2

Begin Training.....
epochs 0 MSE Loss 0.5815340280532837
epochs 10 MSE Loss 0.30040234327316284
Epoch    13: reducing learning rate of group 0 to 9.9990e-04.
epochs 20 MSE Loss 0.1971011906862259
Epoch    25: reducing learning rate of group 0 to 9.9980e-04.
epochs 30 MSE Loss 0.1537417769432068
Epoch    37: reducing learning rate of group 0 to 9.9970e-04.
epochs 40 MSE Loss 0.13426533341407776
Epoch    49: reducing learning rate of group 0 to 9.9960e-04.
epochs 50 MSE Loss 0.127436101436615
Epoch    61: reducing learning rate of group 0 to 9.9950e-04.
epochs 60 MSE Loss 0.12520436942577362
epochs 70 MSE Loss 0.123199462890625
Epoch    73: reducing learning rate of group 0 to 9.9940e-04.
epochs 80 MSE Loss 0.12245701998472214
Epoch    85: reducing learning rate of group 0 to 9.9930e-04.
epochs 90 MSE Loss 0.12258145213127136
Epoch    97: reducing learning rate of group 0 to 9.9920e-04.
epochs 100 MSE Loss 0.12206724286079407
Epoch   109: reducing learning rate of group 0 to 9.9910e-04.


In [29]:
model_3, h1_3,h_2_3 = train_layer(50,
                                  10,
                                  h1_2,
                                  h_2_2,
                                  epochs=n_epochs,
                                  learning_rate = 0.001,
                                  batch_size=bs)
torch.save(model_3.state_dict(), "w_layer_3")
del model_3

Begin Training.....
epochs 0 MSE Loss 0.823908805847168
epochs 10 MSE Loss 0.6994829177856445
Epoch    13: reducing learning rate of group 0 to 9.9990e-04.
epochs 20 MSE Loss 0.5778981447219849
Epoch    25: reducing learning rate of group 0 to 9.9980e-04.
epochs 30 MSE Loss 0.4627338945865631
Epoch    37: reducing learning rate of group 0 to 9.9970e-04.
epochs 40 MSE Loss 0.36975908279418945
Epoch    49: reducing learning rate of group 0 to 9.9960e-04.
epochs 50 MSE Loss 0.29953598976135254
Epoch    61: reducing learning rate of group 0 to 9.9950e-04.
epochs 60 MSE Loss 0.25207841396331787
epochs 70 MSE Loss 0.22157610952854156
Epoch    73: reducing learning rate of group 0 to 9.9940e-04.
epochs 80 MSE Loss 0.20041602849960327
Epoch    85: reducing learning rate of group 0 to 9.9930e-04.
epochs 90 MSE Loss 0.19017481803894043
Epoch    97: reducing learning rate of group 0 to 9.9920e-04.
epochs 100 MSE Loss 0.18281292915344238
Epoch   109: reducing learning rate of group 0 to 9.9910e-04

# DNN

In [30]:
class DNN(nn.Module):

    def __init__(self,d):
        super(DNN, self).__init__()
        self.relu = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()
        self.fc1 = nn.Linear(d, 100,bias=False)  # d is dimension of the input.
        self.fc2 = nn.Linear(100, 50,bias=False)
        self.fc3 = nn.Linear(50, 10,bias=False)
        self.fc4 = nn.Linear(10,2,bias=False)
        

    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        x = self.sigmoid(self.fc4(x))


        return x

In [31]:
dnn_model = DNN(d=257)
model_1 = RBM(257,100)
model_1.load_state_dict(torch.load("w_layer_1"))
model_2 = RBM(100,50)
model_2.load_state_dict(torch.load("w_layer_2"))
model_3 = RBM(50,10)
model_3.load_state_dict(torch.load("w_layer_3"))

<All keys matched successfully>

In [32]:
with torch.no_grad():
    dnn_model.fc1.weight = model_1.feed.weight
    dnn_model.fc2.weight = model_2.feed.weight
    dnn_model.fc3.weight = model_3.feed.weight

In [33]:
from  helpers_dnn import *
from sklearn.preprocessing import StandardScaler

In [34]:
print(torch.cuda.is_available())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

True
cuda:0


In [49]:
def validation_dnn(s1,s2,u,v,smr):

  mixed_signal , speech_signal, music_signal = get_mixed_signal(test_speech,test_music,smr)
  f,t,mixed_spectrum = signal.stft(mixed_signal,window=WINDOW,nperseg=WINDOW_SIZE,noverlap=OVERLAP,nfft=NFFT) # spectrum  of mixed signal
  #mixed_spectrum_ = MinMaxScaler().fit_transform(np.abs(mixed_spectrum))
  new_s1 = (u*s1).cpu().detach().numpy().reshape(257,-1)
  new_s2 = (v*s2).cpu().detach().numpy().reshape(257,-1)

  s1_new = (mixed_spectrum*new_s1) /(new_s1 + new_s2 +0.000001 )
  s2_new = (mixed_spectrum*new_s2) /(new_s1 + new_s2 +0.000001)


  _, speech_estimate =  signal.istft( s1_new,samplerate,window=WINDOW,nperseg=WINDOW_SIZE,noverlap=OVERLAP,nfft=NFFT)
  _, music_estimate =  signal.istft( s2_new,samplerate,window=WINDOW,nperseg=WINDOW_SIZE,noverlap=OVERLAP,nfft=NFFT)

  print("Separation finish ! ....")
  #print(speech_estimate.shape, speech.shape)
  sdr_speech = SDR(speech_estimate[:speech_signal.shape[0]],speech_signal)
  sdr_music = SDR(music_estimate[:music_signal.shape[0]],music_signal)

  print("SDR Speech = {}\n SDR Music = {}".format(sdr_speech,sdr_music))

In [73]:
scaler = MinMaxScaler()
spectrum_s1_ = scaler.fit_transform(np.abs(spectrum_s1).reshape(-1,257))
spectrum_s2_ = MinMaxScaler().fit_transform(np.abs(spectrum_s2.reshape(-1,257)))
Y_mag_ = MinMaxScaler().fit_transform(Y_mag.reshape(-1,257))

s1 = torch.tensor(spectrum_s1_, dtype = torch.float32)
s2 = torch.tensor(spectrum_s2_,dtype=torch.float32)
Yabs = torch.tensor(Y_mag_,dtype=torch.float32)
u,v = gain_params_vec(s1, s2, Yabs)
model = DNN(257)

from torch.optim import LBFGS, Adam,SGD
s1 = s1.to(device)
s2 = s2.to(device)
u = u.to(device)
v = v.to(device)
Yabs = Yabs.to(device)

s1.requires_grad = True
s2.requires_grad = True
u.requires_grad =  True
v.requires_grad =  True
Yabs.requires_grad = False
model = model.to(device)

params = list(model.parameters()) + [u] + [v] + [s1] + [s2]
optimizer= SGD(params, lr=0.001)

In [74]:
L2_LOSS=torch.norm(u*s1 + v*s2 - Yabs,'fro',None)
print('Initial Loss = {}'.format(L2_LOSS))

Initial Loss = 36.16548538208008


In [75]:
def Criteria(s1,s2,Yabs,u,v,i,model,beta,l):
    
        # Feed forward and get energy 1 and 2
    e1 = energy_1(s1[i,:].float(),model)
    e2 = energy_2(s2[i,:].float(),model)

    # Get least square error :
    e_rr=E_err_vec(s1[i,:], s2[i,:], Yabs[i,:],u[i],v[i])

    # Non negative constraint
    R = nonneg_constraint_sum(s1[i,:], s2[i,:], u[i], v[i])
    
    return e1 + e2 + l*e_rr + beta*R

In [ ]:
optimizer= SGD(params, lr=0.005)
N_epochs = 1000

model.train()
qq = []
Yabs = Yabs.to(device)
for e in range(N_epochs):
  loss_e=0.0


  model.fc1.weight.requires_grad = True
  model.fc2.weight.requires_grad = True
  model.fc3.weight.requires_grad = True

  for i in range(s1.shape[1]):

            
          optimizer.zero_grad()

          loss = Criteria(s1,s2,Yabs,u,v,i,model,3,5).to(device)
          loss.backward()
          # fill zeros into the first row of grad
          u.grad.data[:i].fill_(0)
          u.grad.data[i+1:].fill_(0)
          
          v.grad.data[:i].fill_(0)
          v.grad.data[i+1:].fill_(0)
          s1.grad.data[:,:i].fill_(0)
          s1.grad.data[:,i+1:].fill_(0)
          s2.grad.data[:,:i].fill_(0)
          s2.grad.data[:,i+1:].fill_(0)



          optimizer.step()
          loss_e+=loss
  
  if e%5 == 0:    
    print('Epoch {} , Loss = {:.2f}'.format(e,loss_e/s1.shape[1]))
    validation_dnn(s1,s2,u,v,-5)


In [ ]:
SDR Speech = -0.8132213434622194
SDR Music  = 4.1867786565377765

SDR Speech = -0.69
 SDR Music = 4.31

In [2]:
torch.save("model_5",model)


NameError: ignored